In [1]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame
from risk_metrics import Risk_Metrics
import os
import logging

In [2]:
connection = create_connection("../database/crypto_billionairs.db")

In [3]:
headers = ["time", "open", "high", "low", "close", "volume"]

In [4]:
def momentum(df, lag):
    return df.pct_change(periods=lag)

In [5]:
def prepare_files(path, db_connection, header_list):
    
    for file in os.listdir(path):
        
        try:
            
            if file[-3:] == 'txt':
                df = pd.read_csv(f'./{path}/{file}', names = header_list)
             
                df["momentum_1week"] = momentum(df["close"], 10080)
                df["year"] = df["time"].str[:4]
                df["month"] = df["time"].str[5:7]
                df["day"] = df["time"].str[8:10]
    
                # df["buy_indicator"] = 0
                # df.loc[df["momentum_1week"] > 0.025, 'buy_indicator'] = 1
                # df.loc[df["momentum_1week"] < -0.08, 'short_indicator'] = -1
                df_ti = pd.DataFrame()
                df_ti["time"] = df["time"]
                df_ti["open"] = df["open"]
                df_ti["close"] = df["close"]
                df_ti["high"] = df["high"]
                df_ti["low"] = df["low"]
                df_ti["volume"] = df["volume"]
                stock = StockDataFrame.retype(df_ti)
                df["stochastic_oscillator"] = stock.get("kdjk")
                df["relative_strength_index"] = stock.get("rsi_30")
                df["moving_average_convergence_divergence"] = stock.get("macd")
                df["moving_average_convergence_divergence"] = df["moving_average_convergence_divergence"].astype(int)
                df["so_group"] = df["stochastic_oscillator"].astype(str).str[0]
                df["rsi_group"] = df["relative_strength_index"].astype(str).str[0]
                df["technical_indicators"] = df["so_group"] + "/" + df["rsi_group"]
                #df["momentum168"] = momentum(df["close"], 2016)
                df["buy_indicator"] = 0
                df.loc[df["technical_indicators"] == "1/1", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "2/1", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "3/1", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "4/1", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "5/7", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "6/1", 'buy_indicator'] = 1
                df.loc[df["technical_indicators"] == "8/1", 'buy_indicator'] = 1
                
                df["close_indicator"] = 0
                df.loc[df["technical_indicators"].shift(10080) == "1/1", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "2/1", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "3/1", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "4/1", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "5/7", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "6/1", 'close_indicator'] = 2
                df.loc[df["technical_indicators"].shift(10080) == "8/1", 'close_indicator'] = 2
                
                
                short_signals = df.index[df["moving_average_convergence_divergence"] >= 55].tolist()
                df["short_indicator"] = 0
                df["short_indicator"].loc[short_signals] = 1
            
                file_name = str(file).replace("-", "_")
                
                df.to_sql(f'{file_name[:-4]}_complete_history_long_short_rsi', con=db_connection, if_exists="replace", index=False)
                #os.remove(f'{path}/{file}')
        except:
            print(file)

In [6]:
prepare_files("../database/", connection, headers)

C:\Users\janfa\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
class Momentum_RSI_long_short(Strategy):
   
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.buy_init = self.I(init_buy_signal, self.data.df)
        self.close_buy_init = self.I(init_close_long_signal, self.data.df)
        self.short_init = self.I(init_short_signal, self.data.df)
        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.buy_init[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()
            
        elif self.close_buy_init[-1] == 2 and self.position.is_long:
             self.position.close()
             
        elif self.short_init[-1] == -1 and self.position.is_short is False:
            self.position.close()
            self.sell()

In [8]:
def init_buy_signal(trash):
    return df_temp['buy_indicator']

def init_short_signal(trash):
    return df_temp['short_indicator']

def init_close_long_signal(df):
    return df_temp['close_indicator']

In [9]:
def run_backtesting(db_connection):
    
    global df_temp
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "1min_complete_history_long_short_rsi" in name and 'trades' not in name]
    
    df_risk = pd.DataFrame(columns = range(10))
    df_risk.columns = ["table_name", "timehorizon", "return", "annualized_return", "sharpe_ratio_annualized", "sortino_ratio_annualized", "maximum_drawdown", "calmar_ratio_annualized", "trades_count", "win_rate"]
    
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"select * from {table}", db_connection)
        
        df_backtesting = pd.DataFrame()
        df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_temp[['open', 'high', 'low', 'close', 'volume']]
        
        
        bt = Backtest(df_backtesting, Momentum_RSI_long_short, cash=100_000, commission=.001)
        stats = bt.run()
        trades = pd.DataFrame(stats['_trades'])
        trades.to_sql(f"trades_{table}", db_connection, if_exists="replace")
        
        risk = Risk_Metrics(trades, df_temp[10080:], 0)
        risk_metrics_list = [f"{table}", len(df_backtesting), stats["Return [%]"], risk._annualize(stats["Return [%]"]),
                             risk.sharpe_ratio("momentum_1week"), risk.sortino_ratio("momentum_1week"),
                             risk.max_drawdown(), risk.calmar_ratio(stats["Return [%]"]), len(trades), stats["Win Rate [%]"]]
        print(risk_metrics_list)
        #df_risk.append(risk_metrics_list)
        df_risk.loc[len(df_risk)] = risk_metrics_list
        
    
    df_risk.to_sql("cryptocurrencies_risk_metrics_1m_rsi", db_connection, if_exists="replace")

In [10]:
run_backtesting(connection)

<ipython-input-9-357ed40be165>:22: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, Momentum_RSI_long_short, cash=100_000, commission=.001)


['ADA_1min_complete_history_long_short_rsi', 1547195, 1075.246454299314, 367.6690009398903, 22.622989671304467, 22.624261726866006, 0.5705731097342185, 644.3854339913705, 105, 61.904761904761905]
['BAT_1min_complete_history_long_short_rsi', 842235, -57.27654454134983, -36.17661590801409, 0.9389356943921079, 1.1943004896261147, 0.7655581103403659, -47.2552186690701, 46, 56.52173913043478]
['BCH_1min_complete_history_long_short_rsi', 2210868, -69.68036545899764, -16.64131215057932, 0.3326612909047994, 0.4282296222275895, 0.925874540321868, -17.97361459447215, 257, 43.96887159533074]
['BNT_1min_complete_history_long_short_rsi', 1665182, 257.7200527484329, 81.8424844659582, 4.617846370676654, 5.567982183392863, 0.4748569746696964, 172.35186346980927, 114, 50.0]
['BSV_1min_complete_history_long_short_rsi', 1275677, 315.56571017210456, 131.05383251260722, 7.671865433856459, 8.609792519423026, 0.692290317594228, 189.3047312405455, 100, 51.0]
['BTC_1min_complete_history_long_short_rsi', 397947